### CNN algorithm for detection of noraml or pnemonia health conditions

###  loading the training dataset and formatting it

In [1]:
# Deep Learning CNN model to recognize disease
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''

TrainingImagePath='D:\\python\\17810_23812_bundle_archive\\chest_xray\\Training'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)


# Generating the Training Data
training_set= train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 224 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


{'Normal': 0, 'Pneumonia': 1}

### Creating a list of health condition and classes

In [2]:
'''#################### Creating lookup table for all faces ##############################'''
# class_indices have the numeric tag for each health condition
TrainClasses=training_set.class_indices

# Storing the condition and the numeric tag for future reference
ResultMap={}
for ConditionValue,ConditionName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[ConditionValue]=ConditionName

# Saving the face map for future reference
import pickle
with open("D:\\python\\17810_23812_bundle_archive\\chest_xray\\ResultsMap.pkl", 'wb') as f:
    pickle.dump(ResultMap, f, pickle.HIGHEST_PROTOCOL)

print("Mapping of condition and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of conditions
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of condition and its ID {0: 'Normal', 1: 'Pneumonia'}

 The Number of output neurons:  2


### Creating CNN Model

In [3]:
'''######################## Create CNN deep learning model ####################################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()


classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))


'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

### fitting the CNN model

In [4]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=6,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=7)




EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
6/6 [==============================] - 20s 3s/step - loss: 0.8077 - accuracy: 0.5104 - val_loss: 0.6854 - val_accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 20s 3s/step - loss: 0.6798 - accuracy: 0.5260 - val_loss: 0.6581 - val_accuracy: 0.9196
Epoch 3/10
6/6 [==============================] - 19s 3s/step - loss: 0.6480 - accuracy: 0.7604 - val_loss: 0.5951 - val_accuracy: 0.9062
Epoch 4/10
6/6 [==============================] - 19s 3s/step - loss: 0.5821 - accuracy: 0.7812 - val_loss: 0.5070 - val_accuracy: 0.7634
Epoch 5/10
6/6 [==============================] - 20s 3s/step - loss: 0.4886 - accuracy: 0.7969 - val_loss: 0.3694 - val_accuracy: 0.9330
Epoch 6/10
6/6 [==============================] - 19s 3s/step - loss: 0.4064 - accuracy: 0.8490 - val_loss: 0.2911 - val_accuracy: 0.9062
Epoch 7/10
6/6 [==============================] - 19s 3s/step - loss: 0.2956 - accuracy: 0.8854 

### Saving the model

In [5]:
## Saving the model
classifier.save("D:\\python\\17810_23812_bundle_archive\\chest_xray\\classifier.pkl")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: D:\python\17810_23812_bundle_archive\chest_xray\classifier.pkl\assets


### Testing the model on different health condition

In [9]:
'''########################## Making single predictions ############################'''
import numpy as np
from keras.preprocessing import image

testImage='D:\\python\\17810_23812_bundle_archive\\chest_xray\\test\\PNEUMONIA\\person1_virus_7.jpeg'
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  Pneumonia
